In [69]:
import torch 
import torch.nn as nn
import numpy

w = torch.randn(10,11)

u,s,v = torch.svd(w)

recover = u@torch.diag(s)@v.T

full_rank = len(s)
low_rank = 3
print('full rank : ',full_rank,'   low rank : ',low_rank)
x = torch.zeros(full_rank-low_rank)
low_s = torch.cat( (s[:low_rank],x),dim=0)
low_s

full rank :  10    low rank :  3


tensor([5.2156, 5.1459, 3.9245, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000])

### SVD를 해서 복원하면 완전히 같지는 않다!

In [74]:
counts = 0
non_counts = 5
for i in range(10):
    for j in range(11):
        if recover[i][j]==w[i][j]:
            counts+=1
        # else:
        #     non_counts-=1
        #     print(recover[i][j].numpy(),w[i][j].numpy())
    #     if non_counts==0:
    #         break
    # break
counts

8

In [86]:
import torch
import torchvision
import torchvision.transforms as transforms

# 데이터 세트 로드
# train_data = torchvision.datasets.ImageFolder(
#     "data/train",
#     transform=transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ])
# )

# 변환기 정의
def normalize(x):
    return x / x.std()

# 데이터 세트 변환
# train_data = train_data.map(normalize, batch_size=1)

# 모델 정의
class Generator(torch.nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = torchvision.models.resnet18(False)
        self.linear = torch.nn.Linear(512, 3 * 224 * 224)

    def forward(self, x):
        x = self.model(x)
        x = self.linear(x.view(-1, 512))
        x = x.view(-1, 3, 224, 224)
        return x

# 모델 훈련
model = Generator()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

# for epoch in range(100):
#     for data in train_data:
#         img, _ = data
#         img = img.float()
#         output = model(img)
#         loss = criterion(output, img)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

# 모델 평가
# test_data = torchvision.datasets.ImageFolder(
#     "data/test",
#     transform=transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ])
# )

# correct = 0
# total = 0
# for data in test_data:
#     img, gt = data
#     img = img.float()
#     output = model(img)
#     output = output.argmax(dim=1)
#     gt = gt.argmax(dim=1)
#     correct += (output == gt).sum().item()
#     total += len(gt)

# accuracy = correct / total
# print("Accuracy:", accuracy)

# 저등급 적응
def low_rank_adaptation(W, r):
    U, s, V = torch.svd(W)
    
    full_rank = len(s)
    low_rank = r
    print('full rank : ',full_rank,'   low rank : ',low_rank)
    x = torch.zeros(full_rank-low_rank)
    low_s = torch.cat( (s[:low_rank],x),dim=0)
    return U @ torch.diag(low_s) @ V.T

# 모델 매개변수 저등급화
W = model.linear.weight.data
W_low_rank = low_rank_adaptation(W, 100)
model.linear.weight.data = W_low_rank

# 모델 평가
correct = 0
total = 0
# for data in test_data:
#     img, gt = data
#     img = img.float()
#     output = model(img)
#     output = output.argmax(dim=1)
#     gt = gt.argmax(dim=1)
#     correct += (output == gt).sum().item()
#     total += len(gt)

# accuracy = correct / total
# print("Accuracy after low-rank adaptation:", accuracy)


full rank :  512    low rank :  100


In [83]:
# W_low_rank = low_rank_adaptation(W, 100)
# model.linear.weight.data = W_low_rank



torch.Size([512])